In [1]:
#index.sh

In [34]:
import re
import argparse
import document_pb2
import struct
import gzip
import sys
import array
import os
import pymorphy2

In [85]:
PYMORPHY_CACHE = {}
morph = pymorphy2.MorphAnalyzer()

#help funcs
def pymorphy_tokenizer(text):
    global PYMORPHY_CACHE

    for word in text:
        word_hash = hash(word)
        if word_hash not in PYMORPHY_CACHE:
            PYMORPHY_CACHE[word_hash] = morph.parse(word)[0].normal_form            
        yield PYMORPHY_CACHE[word_hash]
        
def union(arr1, arr2):
    m, n, i, j = len(arr1), len(arr2), 0, 0
    result = []
    
    while i < m and j < n: 
        if arr1[i] < arr2[j]: 
            result.append(arr1[i]) 
            i += 1
        elif arr2[j] < arr1[i]: 
            result.append(arr2[j]) 
            j+= 1
        else: 
            result.append(arr2[j]) 
            j += 1
            i += 1
  
    while i < m: 
        result.append(arr1[i]) 
        i += 1
    while j < n: 
        result.append(arr2[j]) 
        j += 1
        
    return result
        
def intersection(arr1, arr2): 
    m, n, i, j = len(arr1), len(arr2), 0, 0
    result = []
    
    while i < m and j < n: 
        if arr1[i] < arr2[j]: 
            i += 1
        elif arr2[j] < arr1[i]: 
            j += 1
        else: 
            result.append(arr2[j]) 
            j += 1
            i += 1
            
    return result

In [86]:
#coders

In [87]:
class Varbyte:
    def __init__(self):
        pass
    
    def pack(self, numbers):
        bytes_list = []
        for number in numbers:
            bytes_list.append(self.encode_number(number))
        return b''.join(bytes_list)
    
    def encode_number(self, number):
        bytes_list = []
        while True:
            bytes_list.insert(0, number % 128)
            if number < 128:
                break
            number = number // 128
        bytes_list[-1] += 128
        return struct.pack('%dB' % len(bytes_list), *bytes_list)
        
    def unpack(self, packed):
        n = 0
        numbers = []
        bytestream = struct.unpack('%dB' % len(packed), packed)
        for byte in bytestream:
            if byte < 128:
                n = 128 * n + byte
            else:
                n = 128 * n + (byte - 128)
                numbers.append(n)
                n = 0
        return numbers
    
#почему-то он не сжимает сильно лучше Varbyte...
class Simple9:
    def __init__(self):
        self.code = {1 : 0, 2 : 1, 3 : 2, 4 : 3, 5 : 4, 7 : 5, 9 : 6, 14 : 7, 28 : 8}
        self.coded = {0 : 1, 1 : 2, 2 : 3, 3 : 4, 4 : 5, 5 : 7, 6 : 9, 7 : 14, 8 : 28}

    def encode_chunk(self, numbers):
        n, length = 0, len(numbers)
        n = int(self.code[length] << 28)
        for i in range(length):
            n += numbers[i] << ((28 // length) * i)
        
        return struct.pack('I', n)
    
    def pack(self, numbers):
        bytes_list = []
        
        n, length = 0, len(numbers)
        while n < length:
            if n + 28 <= length and max(numbers[n:n + 28:]) <= 1:
                bytes_list.append(self.encode_chunk(numbers[n:n + 28:]))
                n += 28
            elif n + 14 <= length and max(numbers[n:n + 14:]) <= 3:
                bytes_list.append(self.encode_chunk(numbers[n:n + 14:]))
                n += 14
            elif n + 9 <= length and max(numbers[n:n + 9:]) <= 7:
                bytes_list.append(self.encode_chunk(numbers[n:n + 9:]))
                n += 9
            elif n + 7 <= length and max(numbers[n:n + 7:]) <= 15:
                bytes_list.append(self.encode_chunk(numbers[n:n + 7:]))
                n += 7
            elif n + 5 <= length and max(numbers[n:n + 5:]) <= 31:
                bytes_list.append(self.encode_chunk(numbers[n:n + 5:]))
                n += 5
            elif n + 4 <= length and max(numbers[n:n + 4:]) <= 127:
                bytes_list.append(self.encode_chunk(numbers[n:n + 4:]))
                n += 4
            elif n + 3 <= length and max(numbers[n:n + 3:]) <= 511:
                bytes_list.append(self.encode_chunk(numbers[n:n + 3:]))
                n += 3
            elif n + 2 <= length and max(numbers[n:n + 2:]) <= 2**14 - 1:
                bytes_list.append(self.encode_chunk(numbers[n:n + 2:]))
                n += 2
            else:
                bytes_list.append(self.encode_chunk([numbers[n]]))
                n += 1
         
        return b''.join(bytes_list)
        
    def unpack(self, packed):
        numbers = []
        n = 0
        stream = struct.unpack('%dI' % (len(packed) // 4), packed)
        for chunk in stream:
            length = self.coded[chunk >> 28]
            for i in range(length):
                n = (chunk >> ((28 // length) * i)) % (1 << (28 // length))
                numbers.append(n)
                
        return numbers

In [88]:
PATHS = ['lenta.ru_159b9f4b-972b-48b1-8ec3-44fbd6be33c4_01.gz']

class DocStreamReader:
    def __init__(self, paths):
        self.paths = paths

    def open_single(self, path):
        return gzip.open(path, 'rb') if path.endswith('.gz') else open(path, 'rb')

    def __iter__(self):
        for path in self.paths:
            with self.open_single(path) as stream:
                while True:
                    sb = stream.read(4)
                    if len(sb) == 0:
                        break

                    size = struct.unpack('i', sb)[0]
                    msg = stream.read(size)
                    doc = document_pb2.document()
                    doc.ParseFromString(msg)
                    yield doc

In [89]:
class IndexCreator:
    def __init__(self, pack_type='varbyte'):
        self.doc_to_url = {}
        self.index = {}
        self.pack_type = pack_type
            
        self.SPLIT_RGX = re.compile(r'\w+', re.U)
    
    def create_index(self, docs):
        for doc_id, doc in enumerate(docs):
            terms = set(pymorphy_tokenizer(re.findall(self.SPLIT_RGX, doc.text.lower())))
            self.doc_to_url[doc_id] = doc.url
            
            for term in terms:
                if term in self.index.keys():
                    self.index[term].append(doc_id)
                else:
                    self.index[term] = array.array('I', [doc_id])
        
        return self.index, self.doc_to_url
    
    def compress_index(self):
        unpacked_size = 0
        packed_size = 0
        
        for term in self.index:
            posting_list = self.index[term]
            
            #храним промежутки, а не сами числа
            unpacked = [posting_list[0]]
            for i in range(len(posting_list) - 1):
                unpacked.append(posting_list[i + 1] - posting_list[i])
            
            packed = b''
            if self.pack_type == 'varbyte':
                packer = Varbyte()
                packed = packer.pack(unpacked)
            elif self.pack_type == 'simple9':
                packer = Simple9()
                packed = packer.pack(unpacked)
                
            unpacked_size += sys.getsizeof(posting_list)
            packed_size += sys.getsizeof(packed)
            self.index[term] = packed
            
        print(unpacked_size)
        print(packed_size)
            
    def save_index(self):
        path = 'index.gz'
        if self.pack_type is not None:
            path = self.pack_type + '_' + path
            
        with gzip.open(path, 'wb') as stream:
            for term in self.index:
                tb = bytes(term, 'utf-8')
                stream.write(struct.pack('I', len(tb)))
                stream.write(tb)
                stream.write(struct.pack('I', len(self.index[term])))
                stream.write(self.index[term])
                
        path = 'docs_url.gz'
        with gzip.open(path, 'wb') as stream:
            for doc_id in self.doc_to_url:
                stream.write(struct.pack('I', doc_id))
                ub = bytes(self.doc_to_url[doc_id], 'utf-8')
                stream.write(struct.pack('I', len(ub)))
                stream.write(ub)

In [90]:
reader = DocStreamReader(PATHS)
# for doc in reader:
#     print(doc.url, len(doc.text))
index_creator = IndexCreator('varbyte')
index, url = index_creator.create_index(reader)

print(index['абсолютно'])
print(url[index['абсолютно'][0]])

index_creator.compress_index()
index_creator.save_index()

array('I', [23, 50, 205, 249, 498, 576, 761, 762, 973, 1023, 1037, 1223])
http://lenta.ru/culture/2002/12/26/mult/
2514156
1058551


In [91]:
#make_dict.sh

In [92]:
test = [23, 27, 155, 44, 249, 78, 185, 1, 211, 50, 14, 186]
packer = Varbyte()
packed = packer.pack(test)
unpacked = packer.unpack(packed)
print(unpacked)

[23, 27, 155, 44, 249, 78, 185, 1, 211, 50, 14, 186]


In [93]:
#search.sh

In [94]:
class Index:
    def __init__(self):
        path = 'varbyte_index.gz'
        self.pack_type = 'varbyte'
        
        if os.path.exists('simple9_index.gz'):
            path = 'simple9_index.gz'
            self.pack_type = 'simple9'
            
        self.index = {}
        with gzip.open(path, 'rb') as stream:
            while True:
                sb = stream.read(4)
                if len(sb) == 0:
                    break
                    
                size = struct.unpack('I', sb)[0]
                term = stream.read(size).decode('utf-8')
                
                sb = stream.read(4)
                size = struct.unpack('I', sb)[0]
                posting_list = stream.read(size)
                
                self.index[term] = posting_list
    
    def urls(self):
        path = 'docs_url.gz'
        
        self.docs_url = {}
        with gzip.open(path, 'rb') as stream:
            while True:
                sb = stream.read(4)
                if len(sb) == 0:
                    break
                    
                doc_id = struct.unpack('I', sb)[0]
                
                sb = stream.read(4)
                size = struct.unpack('I', sb)[0]
                url = stream.read(size).decode('utf-8')
                
                self.docs_url[doc_id] = url
        
        return self.docs_url
        
    def __getitem__(self, item):
        item = next(pymorphy_tokenizer([item]))
        packed = self.index[item]
        posting_list = []
        
        if self.pack_type == 'varbyte':
            packer = Varbyte()
            posting_list = packer.unpack(packed)
        elif self.pack_type == 'Simple9':
            packer = Simple9()
            posting_list = packer.unpack(packed)
            
        for i in range(len(posting_list) - 1):
            posting_list[i + 1] += posting_list[i]
            
        return posting_list


In [106]:
class Parser:
    def __init__(self, string, dictionary = None):
        self.string = string
        self.index = 0
        self.dictionary = dictionary

    def get_value(self):
        value = self.parse_expression()
        self.skip_whitespace()

        return value

    def peek(self):
        return self.string[self.index:self.index + 1]

    def has_next(self):
        return self.index < len(self.string)

    def is_next(self, value):
        return self.string[self.index:self.index + len(value)] == value

    def pop_if_next(self, value):
        if self.is_next(value):
            self.index += len(value)
            return True
        return False

    def pop_expected(self, value):
        if not self.pop_if_next(value):
            raise Exception("Expected '" + value + "' at index " + str(self.index))

    def skip_whitespace(self):
        while self.has_next():
            if self.peek() in ' \t\n\r':
                self.index += 1
            else:
                return

    def parse_expression(self):
        return self.parse_or()
    
    def parse_or(self):
        values = [self.parse_and()]
        
        while True:
            self.skip_whitespace()
            char = self.peek()
            
            if char == '|':
                self.index += 1
                values.append(self.parse_and())
            else:
                break
        
        value = values[0]
        
        for factor in values[1::]:
            value = union(value, factor)
        return value

    def parse_and(self):
        values = [self.parse_parenthesis()]
            
        while True:
            self.skip_whitespace()
            char = self.peek()
                
            if char == '&':
                self.index += 1
                values.append(self.parse_parenthesis())
            else:
                break
                     
        value = values[0]
        
        for factor in values[1::]:
            value = intersection(value, factor)
        return value

    def parse_parenthesis(self):
        self.skip_whitespace()
        char = self.peek()
        
        if char == '(':
            self.index += 1
            value = self.parse_expression()
            self.skip_whitespace()
            
            if self.peek() != ')':
                raise Exception("No closing parenthesis found at character " + str(self.index))
            self.index += 1
            return value
        else:
            return self.parse_not()

    def parse_arguments(self):
        args = []
        self.skip_whitespace()
        self.popExpected('(')
        while not self.pop_if_next(')'):
            self.skip_whitespace()
            if len(args) > 0:
                self.pop_expected(',')
                self.skip_whitespace()
            args.append(self.parse_expression())
            self.skip_whitespace()
        return args

    def parse_not(self):
        self.skip_whitespace()
        char = self.peek()
        
        if char == '!':
            self.index += 1
            return self.parse_parenthesis()
        else:
            return self.parse_value()

    def parse_value(self):
        self.skip_whitespace()
        var = []
        while self.has_next():
            char = self.peek()
            
            if char.lower() in 'абвгдеёжзийклмнопрстуфхцчшщъыьэюяabcdefghijklmnopqrstuvwxyz0123456789':
                var.append(char)
                self.index += 1
            else:
                break
                
        var = ''.join(var)
        return self.dictionary[var]

In [107]:
index = Index()
urls = index.urls()
print(index['абсолютно'])
print(index['редкость'])

[23, 50, 205, 249, 498, 576, 761, 762, 973, 1023, 1037, 1223]
[443]


In [116]:
test1 = 'абсолютно'
test2 = 'слова & редкость'
test3 = '(слово | абсолютно) & редкость'
test4 = '(слово & абсолютно) | редкости'

print(Parser(test1, index).get_value())
print(Parser(test2, index).get_value())
print(Parser(test3, index).get_value())
print(Parser(test4, index).get_value())

test5 = '(словом & редкостью) | (словами & абсолютно) | абсолютно'
result = [urls[i] for i in Parser(test5, index).get_value()]
print(result)

[23, 50, 205, 249, 498, 576, 761, 762, 973, 1023, 1037, 1223]
[443]
[443]
[23, 50, 205, 249, 443, 498, 576, 761, 762, 1023, 1223]
['http://lenta.ru/culture/2002/12/26/mult/', 'http://lenta.ru/conf/tsepliaeva', 'http://lenta.ru/sport/2003/12/09/prize/', 'http://lenta.ru/news/2008/01/21/lutcenko/', 'http://lenta.ru/articles/2015/06/16/jfriske/', 'http://lenta.ru/news/2008/02/20/u2/', 'http://lenta.ru/most/2001/04/21/gusinsky/', 'http://lenta.ru/news/2014/07/29/osce1/', 'http://lenta.ru/articles/2014/11/18/granprirussia/', 'http://lenta.ru/articles/2006/09/04/players/', 'http://lenta.ru/news/2006/04/17/techno/', 'http://lenta.ru/news/2012/10/05/denin1/', 'http://lenta.ru/columns/2012/05/16/georgia/']
